In [1]:
import pandas as pd
import sklearn
import numpy as np
from sklearn.model_selection import cross_val_score
import xgboost as xgb

In [2]:
data = pd.read_csv("/data/share/lab04data/lab04_train.csv")

In [3]:
data = data.dropna(subset=['TARGET'])
y_train = data.TARGET
data.drop("TARGET", inplace=True, axis=1)

In [4]:
def get_columns(data, typ):
    return [x for x in data.columns if data[x].dtype.name==typ]

In [5]:
def preprocess(data):
    data.drop("Unnamed: 0", inplace=True, axis=1)
    data.set_index("ID", inplace=True)
    data[get_columns(data, 'float64')].fillna(0, inplace=True)
    data[get_columns(data, 'int64')].fillna(0, inplace=True)
    data[get_columns(data, 'object')].fillna("unk", inplace=True)

In [6]:
preprocess(data)

/opt/anaconda/envs/bd9/lib/python3.6/site-packages/pandas/core/frame.py:3787: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  downcast=downcast, **kwargs)


In [7]:
num_features = get_columns(data, 'int64') + get_columns(data, 'float64')

In [8]:
cross_val_score(xgb.XGBClassifier(),data[num_features], y_train, cv=5)

/opt/anaconda/envs/bd9/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/opt/anaconda/envs/bd9/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/opt/anaconda/envs/bd9/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/opt/anaconda/envs/bd9/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is a

array([0.91894504, 0.91880349, 0.91891134, 0.91892692, 0.91897369])

##### The main goal of this task is to achive roc-auc score 0.83. We can do it without categorical features, so we just drop them.

In [9]:
model = xgb.XGBClassifier().fit(data[num_features], y_train)

In [10]:
data_test = data = pd.read_csv("/data/share/lab04data/lab04_test.csv")

In [11]:
preprocess(data_test)

/opt/anaconda/envs/bd9/lib/python3.6/site-packages/pandas/core/frame.py:3787: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  downcast=downcast, **kwargs)


In [12]:
res = pd.DataFrame(data=data_test.index)
res['target'] = model.predict_proba(data_test[num_features])[:,1]
res.columns = ['id', 'target']

In [13]:
res.to_csv("lab04.csv", sep='\t', index=False)